#### Fix the Error Proposed Value

In [ ]:
import os
import json
import math
import shutil
import time

base_dir = "../results"
backup_dir = "../results_backup_" + time.strftime("%Y%m%d_%H%M%S")
log_file = "../logs/price_scale_fixes_log.txt"

def fix_price_scale(price_list):
    if not price_list or len(price_list) <= 1:
        return price_list, False
    fixed_list = price_list.copy()
    reference = fixed_list[0]
    if reference == 0:
        for i in range(1, len(fixed_list)):
            if fixed_list[i] != 0:
                reference = fixed_list[i]
                break
        if reference == 0:
            return fixed_list, False
    ref_magnitude = math.floor(math.log10(abs(reference)))
    changed = False
    for i in range(1, len(fixed_list)):
        current = fixed_list[i]
        if current == 0:
            continue
        curr_magnitude = math.floor(math.log10(abs(current)))
        if abs(ref_magnitude - curr_magnitude) > 2:
            changed = True
            if ref_magnitude > curr_magnitude:
                scale_factor = 10 ** (ref_magnitude - curr_magnitude)
                fixed_list[i] = current * scale_factor
            else:
                scale_factor = 10 ** (curr_magnitude - ref_magnitude)
                fixed_list[i] = current / scale_factor
    return fixed_list, changed

os.makedirs(backup_dir, exist_ok=True)

total_files = 0
modified_files = 0

with open(log_file, 'w', encoding='utf-8') as log:
    log.write("Fixing price scale inconsistencies in JSON files\n")
    log.write("=" * 80 + "\n")
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.json'):
                total_files += 1
                file_path = os.path.join(root, file)
                try:
                    backup_path = os.path.join(backup_dir, os.path.relpath(file_path, base_dir))
                    os.makedirs(os.path.dirname(backup_path), exist_ok=True)
                    shutil.copy2(file_path, backup_path)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if "seller_price_offers" in data:
                        original_offers = data["seller_price_offers"]
                        fixed_offers, was_modified = fix_price_scale(original_offers)
                        if was_modified:
                            modified_files += 1
                            data["seller_price_offers"] = fixed_offers
                            with open(file_path, 'w', encoding='utf-8') as f:
                                json.dump(data, f, indent=2, ensure_ascii=False)
                            log.write(f"Modified: {file_path}\n")
                            log.write(f"Original: {original_offers}\n")
                            log.write(f"Fixed: {fixed_offers}\n")
                            log.write("-" * 80 + "\n")
                except Exception as e:
                    log.write(f"Error processing {file_path}: {str(e)}\n")
    log.write("\nSummary:\n")
    log.write(f"Total files processed: {total_files}\n")
    log.write(f"Files modified: {modified_files}\n")
    log.write(f"Backup created at: {backup_dir}\n")

print(f"Process completed. {total_files} files processed, {modified_files} files modified.")
print(f"Full backup created at: {backup_dir}")
print(f"See {log_file} for details of changes made.")

#### Check the Anomaly of Price Increase

In [ ]:
import os
import json
from collections import defaultdict
import time
import shutil

def mark_anomalous_data_with_error():
    base_dir = "../results"
    backup_dir = "../results_backup_" + time.strftime("%Y%m%d_%H%M%S")
    log_file = "../logs/data_error_tag_log.txt"
    
    os.makedirs(backup_dir, exist_ok=True)
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    
    seller_models = ["seller_qwen2.5-0.5b-instruct","seller_qwen2.5-1.5b-instruct","seller_qwen2.5-3b-instruct","seller_qwen2.5-7b-instruct", "seller_qwen2.5-14b-instruct","seller_qwen2.5-32b-instruct"]
    buyer_models = ["qwen2.5-0.5b-instruct", "qwen2.5-1.5b-instruct", "qwen2.5-3b-instruct", "qwen2.5-7b-instruct", "qwen2.5-14b-instruct", "qwen2.5-32b-instruct"]
    budgets = ["budget_low", "budget_mid", "budget_high", "budget_wholesale", "budget_retail"]
    products = range(1, 101)
    
    total_files = 0
    modified_files = 0
    
    with open(log_file, 'w', encoding='utf-8') as log:
        log.write("Adding data_error flag to anomalous JSON files\n")
        log.write("=" * 80 + "\n")
        
        for seller_model in seller_models:
            for buyer_model in buyer_models:
                for product_num in products:
                    for budget in budgets:
                        json_path = os.path.join(
                            base_dir, seller_model, buyer_model, f"product_{product_num}", budget, f"product_{product_num}_exp_0.json"
                        )
                        if os.path.exists(json_path):
                            total_files += 1
                            try:
                                # Backup first
                                backup_path = os.path.join(backup_dir, os.path.relpath(json_path, base_dir))
                                os.makedirs(os.path.dirname(backup_path), exist_ok=True)
                                shutil.copy2(json_path, backup_path)
                                
                                # Read data
                                with open(json_path, 'r', encoding='utf-8') as f:
                                    data = json.load(f)
                                
                                # Check for price increase anomaly
                                is_anomalous = False
                                anomaly_details = ""
                                
                                if "seller_price_offers" in data and isinstance(data["seller_price_offers"], list) and len(data["seller_price_offers"]) > 1:
                                    offers = data["seller_price_offers"]
                                    if offers[-1] > offers[0]:
                                        is_anomalous = True
                                        anomaly_details = f"price increase: {offers[0]} -> {offers[-1]}"
                                
                                # Add data_error flag if anomalous
                                if is_anomalous:
                                    data["data_error"] = True
                                    modified_files += 1
                                    
                                    # Write modified data back
                                    with open(json_path, 'w', encoding='utf-8') as f:
                                        json.dump(data, f, indent=2, ensure_ascii=False)
                                    
                                    
                            except Exception as e:
                                log.write(f"Error processing {json_path}: {str(e)}\n")
    
    print(f"Process completed. {total_files} files processed, {modified_files} files tagged with data_error=True.")
    print(f"Full backup created at: {backup_dir}")
    print(f"See {log_file} for details of changes made.")

# Run the function
mark_anomalous_data_with_error()

#### Remove the case that reach MAX_TURN

In [ ]:
import os
import json
import shutil

base_dir = "../results"
target_dir = "../error_data/max_turn"
seller_models = ["seller_qwen2.5-0.5b-instruct","seller_qwen2.5-1.5b-instruct","seller_qwen2.5-3b-instruct","seller_qwen2.5-7b-instruct", "seller_qwen2.5-14b-instruct","seller_qwen2.5-32b-instruct"]
buyer_models = ["qwen2.5-0.5b-instruct", "qwen2.5-1.5b-instruct", "qwen2.5-3b-instruct", "qwen2.5-7b-instruct", "qwen2.5-14b-instruct", "qwen2.5-32b-instruct"]
budgets = ["budget_low", "budget_mid", "budget_high", "budget_wholesale", "budget_retail"]
products = range(1, 101)

moved_count = 0

for seller_model in seller_models:
    for buyer_model in buyer_models:
        for product_num in products:
            for budget in budgets:
                rel_path = os.path.join(
                    seller_model, buyer_model, f"product_{product_num}", budget, f"product_{product_num}_exp_0.json"
                )
                json_path = os.path.join(base_dir, rel_path)
                if os.path.exists(json_path):
                    try:
                        with open(json_path, "r", encoding="utf-8") as f:
                            data = json.load(f)
                        if data.get("negotiation_result") == "max_turns_reached":
                            # 目标路径
                            dest_path = os.path.join(target_dir, rel_path)
                            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
                            shutil.move(json_path, dest_path)
                            moved_count += 1
                            print(f"Moved: {json_path} -> {dest_path}")
                    except Exception as e:
                        print(f"Error processing {json_path}: {e}")